In [51]:
import discogs_client

In [52]:
def readUserToken():
    try:
        with open("user_token.env", "r") as file:
            content = file.read()
            return content
    except FileNotFoundError:
        return {}
        
d = discogs_client.Client('dardok_app/0.1', user_token='{}'.format(readUserToken().rstrip()))

In [53]:
artist_ids = "5531548"
min_average = 4.8

In [55]:
# COMMON FUNCTIONS

def getArtistsIdsList(artists_ids):
    """ A function to convert elements to integers """
    artist_id_list = [int(artist_id) for artist_id in artist_ids.split(',')]
    return artist_id_list

def getFilteredReleaseListFromArtist(artist):
    """ A function that returns all the filtered releases from artist """
    filtered_release_list = []
    print()
    print("Processing artist: {} {}...".format(artist.name, artist.url))
    for k in range(1, result_artist.releases.pages+1):
        print()
        print("Page: {}/{}".format(k,result_artist.releases.pages))
        print(result_artist.releases.page(k))
        for item in result_artist.releases.page(k):
            try:
                # print()
                # print(item)
                release = d.release(item.id)  
                # print()
                # print(release)
                # print(release.url)
                if release.community.rating.average >= min_average and release.formats[0]['name']=='Vinyl' and 'Electronic' in release.genres:
                    filtered_release_list.append(release)
            except Exception as e:
                print()
                print("An exception of type {} occurred: {}".format(type(e).__name__, e))
                
    return filtered_release_list

def funcYear(e):
    """ A function that returns the 'year' value """
    return e.year

def remove_duplicates(input_list):
    """ A function to remove duplicates from a release input_list by id """
    unique_list = []
    seen_ids = set()

    for item in input_list:
        item_id = item.id
        if item_id not in seen_ids:
            seen_ids.add(item_id)
            unique_list.append(item)

    return unique_list

def get_artists(release):
    """ A function to return an artists list from release """
    artists = []
    for artist in release.artists:
        artists.append(artist.name)
    return artists

def get_labels(release):
    """ A function to return an labels list from release """
    labels = []
    for label in release.labels:
        labels.append(label.name)
    return labels

def writeOutputHtml(artist, release_list):
    """ A function to write output as HTML """

    htmlString = "<html>\n<head>\n<title>Filtered Releases for artist {} - id: {}</title>\n</head>\n \
                    <body>\n \
                        <h1>Filtered Releases for label {} - id: {}</h1>\n \
                        <h2>Url: <a href='{}' target='_blank' rel='noopener noreferrer'>{}</a><h2>\n \
                        <br> \
                        Minimum community rating average filter: <span style='color: #ff0000'>{}</span> \n \
                        <br> \
                        Number of results: {} \
                    ".format(artist.name, artist.id, artist.name, artist.id, artist.url, artist.url, min_average, len(release_list))

    htmlString += "<ul style='list-style: none;'>\n"
    k = 0
    for release in release_list:
        htmlString += "<br>\n"
        k += 1
        htmlString += "<span style='color: #5f5f5f'>Result {}/{}</span>\n".format(k, len(release_list)) 
        htmlString += "<li><span style='color: #9b9b9b'>catno:</span> {}</li>\n".format(release.data['labels'][0]['catno'])
        htmlString += "<li><span style='color: #9b9b9b'>Title:</span> {}</li>\n".format(release.title)
        artists = get_artists(release)
        htmlString += "<li><span style='color: #9b9b9b'>Artists:</span> {}</li>\n".format(artists)
        htmlString += "<li><span style='color: #9b9b9b'>Year:</span> {}</li>\n".format(release.year)
        labels = get_labels(release)
        htmlString += "<li><span style='color: #9b9b9b'>Labels:</span> {}</li>\n".format(labels)
        htmlString += "<li><span style='color: #9b9b9b'>Genres:</span> {}</li>\n".format(release.genres)
        htmlString += "<li><span style='color: #9b9b9b'>Styles:</span> {}</li>\n".format(release.styles)
        htmlString += "<li><span style='color: #9b9b9b'>Formats:</span> {}</li>\n".format(release.formats)
        htmlString += "<li><span style='color: #9b9b9b'>Community rating average/count:</span> <span style='color: #ff0000'>{}</span>/<span style='color: #ff8000'>{}</span></li>\n".format(release.community.rating.average, release.community.rating.count)
        htmlString += "<li><span style='color: #9b9b9b'>Url:</span> <a href='{}' target='_blank' rel='noopener noreferrer'>{}</a></li>\n".format(release.url, release.url)

    htmlString += "</ul>\n"
    
    htmlString += "</body>\n \
                  </html>"

    file_name = "{}.html".format(artist.url.lstrip('https://www.discogs.com/artist/'))
    
    with open("./outputHtml/artists/{}".format(file_name), "w") as file:
        file.write(htmlString)

    print()
    print("Written HTML output file to ./outputHtml/artists/{}".format(file_name))

In [57]:
artists_id_list = getArtistsIdsList(artist_ids)
for artist_id in artists_id_list:
    result_artist = d.artist(artist_id)
    filtered_release_list = getFilteredReleaseListFromArtist(result_artist)
    filtered_release_list.sort(reverse=True, key=funcYear)
    unique_filtered_release_list = remove_duplicates(filtered_release_list)
    writeOutputHtml(result_artist, unique_filtered_release_list)
print()
print("Process finished")


Processing artist: Verraco https://www.discogs.com/artist/5531548-Verraco...

Page: 1/1
[<Release 9556777 'Resistir'>, <Master 1961887 'New Army of Androgynes'>, <Release 11488886 "Don't Kill'em All">, <Release 18238102 'RA.671'>, <Master 1831655 'Grial'>, <Release 20855698 'Grial Remixes'>, <Release 28342054 'Escándaloo'>, <Master 1852017 'Megapunk (Remixes)'>, <Release 25257778 'Herrensauna Vol. 2'>, <Master 2781065 'Pliegues'>, <Master 2306245 'Tresor 30'>, <Release 12136071 'Medellín Rave Society'>, <Release 12542203 'Curated by DSH #099'>, <Release 14908504 'Música Urgente'>, <Release 14908647 'Despertar: Chile'>, <Release 18952612 'Horizontes - Vol. 4'>, <Master 2306245 'Tresor 30'>, <Release 23906087 'Desire Path'>, <Release 24936322 'no pare, sigue sigue'>, <Release 27287130 'Atlantic Stereo'>, <Release 27437412 'Desire Path'>, <Release 29356837 'Fact Mix 850'>, <Master 2872324 'Fabric Presents Nicola Cruz'>, <Master 3079121 'Fabric Selects I'>, <Release 29068300 'Los Totes 2'